In [1]:
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import numpy as np

In [2]:
#Now we define a new dataframe called data from yfinance which will contain the stock price of apple on a specified date range.
# Define the stock ticker and date range
ticker = 'GOOG'
start_date = '2010-04-01'
end_date = '2024-11-23'

# Fetch the data using yfinance
data = yf.download(ticker, start=start_date, end=end_date)

# Display the DataFrame
data

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2010-04-01,14.230411,14.282715,14.085952,14.166899,14.132193,84423257
2010-04-05,14.219203,14.318331,14.171880,14.221943,14.187102,76345092
2010-04-06,14.183337,14.218954,14.082216,14.152453,14.117783,82712870
2010-04-07,14.129539,14.165654,13.994047,14.035890,14.001505,103626969
2010-04-08,14.030411,14.193051,13.948966,14.134271,14.099645,78191988
...,...,...,...,...,...,...
2024-11-18,174.955002,176.910004,174.419998,176.800003,176.800003,18725400
2024-11-19,175.235001,180.169998,175.115997,179.580002,179.580002,15392900
2024-11-20,178.830002,179.110001,175.330002,177.330002,177.330002,15729800


In [3]:
import numpy as np
data['VWAP'] = (data['Close'] * data['Volume']).cumsum() / data['Volume'].cumsum()

# Calculate the log difference for 'Open', 'Close', 'High', 'Low', 'Volume'
columns_to_calculate = ['Open', 'Close', 'High', 'Low', 'Volume', 'VWAP']
for column in columns_to_calculate:
    data[f'log_diff_{column}'] = np.log(data[column] / data[column].shift(1))

data.loc[:, 'Target'] = (data['Close'].shift(-1) > data['Close']).astype(int)


# List of columns to calculate logarithms for
columns_to_log = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'VWAP']

# Ensure the columns exist in the DataFrame before processing
existing_columns = [col for col in columns_to_log if col in data.columns]

# Add new columns for logarithms
for column in existing_columns:
    data[f'log_{column}'] = np.log(data[column])

In [4]:
horizons = [2,5,60,250,1000]
new_predictors = []
data['Log_Returns'] = np.log(data['Close'] / data['Close'].shift(1))
for horizon in horizons:
    rolling_averages = data.rolling(horizon).mean()
    
    ratio_column = f"Close_Ratio_{horizon}"
    data[ratio_column] = data["Close"] / rolling_averages["Close"]
    
    trend_column = f"Trend_{horizon}"
    data[trend_column] = data.shift(1).rolling(horizon).sum()["Target"]
    data[f'Volatility_{horizon}'] = data['Log_Returns'].rolling(window=horizon).std()

    new_predictors+= [ratio_column, trend_column]

# Ensure 'Close' contains the daily closing prices


# Calculate rolling volatility (e.g., 20-day rolling window)
# Remove rows with any null values
data = data.dropna()
data

,Open,High,Low,Close,Adj Close,Volume,VWAP,log_diff_Open,log_diff_Close,log_diff_High,...,Volatility_5,Close_Ratio_60,Trend_60,Volatility_60,Close_Ratio_250,Trend_250,Volatility_250,Close_Ratio_1000,Trend_1000,Volatility_1000
Date,,,,,,,,,,,,,,,,,,,,,
2014-03-24,29.494200,29.511883,28.541767,28.840151,28.769499,121939352,16.600681,-0.018507,-0.021453,-0.020656,...,0.014040,0.990601,31.0,0.013405,1.189404,125.0,0.013772,1.671059,517.0,0.016118
2014-03-25,29.041147,29.136787,28.567921,28.859825,28.789124,96769361,16.611799,-0.015480,0.000682,-0.012791,...,0.008811,0.990694,31.0,0.013392,1.188477,126.0,0.013746,1.670782,517.0,0.016117
2014-03-26,28.941769,29.179876,28.181868,28.193575,28.124506,103586819,16.623033,-0.003428,-0.023356,0.001478,...,0.010992,0.967636,30.0,0.013745,1.159434,126.0,0.013814,1.630885,517.0,0.016134
2014-03-27,28.322241,28.322241,27.570307,27.846546,27.778328,262719,16.623061,-0.021638,-0.012385,-0.029832,...,0.009556,0.955609,30.0,0.013800,1.143678,125.0,0.013834,1.609525,517.0,0.016137
2014-03-28,27.983171,28.243956,27.857019,27.922838,27.854433,824257,16.623148,-0.012044,0.002736,-0.002768,...,0.012132,0.958222,30.0,0.013743,1.145367,125.0,0.013808,1.612649,517.0,0.016136
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-18,174.955002,176.910004,174.419998,176.800003,176.800003,18725400,39.989554,-0.003908,0.016596,0.005839,...,0.016599,1.062722,37.0,0.014905,1.104358,150.0,0.016636,1.371619,543.0,0.019136
2024-11-19,175.235001,180.169998,175.115997,179.580002,179.580002,15392900,40.000807,0.001599,0.015602,0.018260,...,0.018521,1.078174,37.0,0.015023,1.120576,150.0,0.016660,1.392215,543.0,0.019138
2024-11-20,178.830002,179.110001,175.330002,177.330002,177.330002,15729800,40.012120,0.020308,-0.012608,-0.005901,...,0.018108,1.063500,37.0,0.015068,1.105507,149.0,0.016672,1.373821,543.0,0.019134


In [5]:
features = ['Log_Returns','Close_Ratio_2', 'Trend_2', 'Close_Ratio_5', 'Trend_5','Close_Ratio_60', 'Trend_60', 'Volatility_60',
       'Close_Ratio_250', 'Trend_250', 'Volatility_250', 'Close_Ratio_1000',
       'Trend_1000', 'Volatility_1000', 'log_Open', 'log_High', 'log_Low',
       'log_Close', 'log_Adj Close', 'log_Volume', 'log_VWAP']
target = ['Target']

In [6]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import numpy as np

# Assuming `data` is your DataFrame
features = features
target = ['Target']

# Split data into training (80%) and testing (20%) sets
split_index = int(len(data) * 0.8)
train_data = data.iloc[:split_index]
test_data = data.iloc[split_index:]

# Separate features (X) and target (y)
X_train = train_data[features]
y_train = train_data[target].values.ravel()
X_test = test_data[features]
y_test = test_data[target].values.ravel()

# Initialize and train the Random Forest model
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)

# Predict probabilities for the test set
y_prob = rf_model.predict_proba(X_test)[:, 1]  # Probability for the positive class

# Custom threshold for classification
custom_threshold = 0.4  # Change this to your desired threshold
y_pred = (y_prob >= custom_threshold).astype(int)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Print evaluation results
print(f"Accuracy: {accuracy:.2f}")
print("\nConfusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(class_report)

# Optional: Display feature importances
feature_importances = pd.DataFrame({
    'Feature': features,
    'Importance': rf_model.feature_importances_
}).sort_values(by='Importance', ascending=False)

print("\nFeature Importances:")
print(feature_importances)


Accuracy: 0.50

Confusion Matrix:
[[ 75 166]
 [103 194]]

Classification Report:
              precision    recall  f1-score   support

           0       0.42      0.31      0.36       241
           1       0.54      0.65      0.59       297

    accuracy                           0.50       538
   macro avg       0.48      0.48      0.47       538
weighted avg       0.49      0.50      0.49       538


Feature Importances:
             Feature  Importance
19        log_Volume    0.068814
3      Close_Ratio_5    0.065490
5     Close_Ratio_60    0.061658
1      Close_Ratio_2    0.061517
0        Log_Returns    0.061348
7      Volatility_60    0.057173
11  Close_Ratio_1000    0.056760
13   Volatility_1000    0.055073
10    Volatility_250    0.054480
8    Close_Ratio_250    0.053317
14          log_Open    0.047918
20          log_VWAP    0.047418
15          log_High    0.046700
16           log_Low    0.045937
18     log_Adj Close    0.045456
17         log_Close    0.045119
9        

In [7]:
# Reset the index to make 'Date' a column
data = data.reset_index()

# Extract only the date part and convert to string format
data['Date'] = data['Date'].dt.date.astype(str)
data

,Date,Open,High,Low,Close,Adj Close,Volume,VWAP,log_diff_Open,log_diff_Close,...,Volatility_5,Close_Ratio_60,Trend_60,Volatility_60,Close_Ratio_250,Trend_250,Volatility_250,Close_Ratio_1000,Trend_1000,Volatility_1000
0,2014-03-24,29.494200,29.511883,28.541767,28.840151,28.769499,121939352,16.600681,-0.018507,-0.021453,...,0.014040,0.990601,31.0,0.013405,1.189404,125.0,0.013772,1.671059,517.0,0.016118
1,2014-03-25,29.041147,29.136787,28.567921,28.859825,28.789124,96769361,16.611799,-0.015480,0.000682,...,0.008811,0.990694,31.0,0.013392,1.188477,126.0,0.013746,1.670782,517.0,0.016117
2,2014-03-26,28.941769,29.179876,28.181868,28.193575,28.124506,103586819,16.623033,-0.003428,-0.023356,...,0.010992,0.967636,30.0,0.013745,1.159434,126.0,0.013814,1.630885,517.0,0.016134
3,2014-03-27,28.322241,28.322241,27.570307,27.846546,27.778328,262719,16.623061,-0.021638,-0.012385,...,0.009556,0.955609,30.0,0.013800,1.143678,125.0,0.013834,1.609525,517.0,0.016137
4,2014-03-28,27.983171,28.243956,27.857019,27.922838,27.854433,824257,16.623148,-0.012044,0.002736,...,0.012132,0.958222,30.0,0.013743,1.145367,125.0,0.013808,1.612649,517.0,0.016136
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2683,2024-11-18,174.955002,176.910004,174.419998,176.800003,176.800003,18725400,39.989554,-0.003908,0.016596,...,0.016599,1.062722,37.0,0.014905,1.104358,150.0,0.016636,1.371619,543.0,0.019136
2684,2024-11-19,175.235001,180.169998,175.115997,179.580002,179.580002,15392900,40.000807,0.001599,0.015602,...,0.018521,1.078174,37.0,0.015023,1.120576,150.0,0.016660,1.392215,543.0,0.019138
2685,2024-11-20,178.830002,179.110001,175.330002,177.330002,177.330002,15729800,40.012120,0.020308,-0.012608,...,0.018108,1.063500,37.0,0.015068,1.105507,149.0,0.016672,1.373821,543.0,0.019134
2686,2024-11-21,175.455002,175.580002,165.309998,169.240005,169.240005,38839400,40.038401,-0.019053,-0.046695,...,0.026496,1.014502,37.0,0.016209,1.054257,149.0,0.016920,1.310340,542.0,0.019181


In [8]:
df2 = pd.read_csv("stock_news_sentiment_analysis_results_GOOG.csv")
df3 = df2.drop(['Ticker'],axis=1)
df = df3.drop(['weighted positive sentiment score', 'weighted negative sentiment score', 'weighted neutral sentiment score'],axis=1)
df

,Date,weighted compound sentiment score
0,2011-05-16,0.261700
1,2011-05-26,0.027200
2,2011-06-14,-0.449700
3,2011-06-16,-0.350300
4,2011-06-17,0.431000
...,...,...
1658,2024-11-05,0.165292
1659,2024-11-06,0.411276
1660,2024-11-07,0.612464
1661,2024-11-08,0.322299


In [9]:
# Merge df and data on the 'Date' column
# This performs a left join, keeping all rows from df and adding matching rows from data
df = pd.merge(df, data, on='Date', how='left')

# Display the modified df
df

,Date,weighted compound sentiment score,Open,High,Low,Close,Adj Close,Volume,VWAP,log_diff_Open,...,Volatility_5,Close_Ratio_60,Trend_60,Volatility_60,Close_Ratio_250,Trend_250,Volatility_250,Close_Ratio_1000,Trend_1000,Volatility_1000
0,2011-05-16,0.261700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2011-05-26,0.027200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2011-06-14,-0.449700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2011-06-16,-0.350300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2011-06-17,0.431000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1658,2024-11-05,0.165292,170.830002,172.097000,170.369995,171.410004,171.410004,12518300.0,39.874694,-0.002397,...,0.018507,1.042915,38.0,0.013779,1.081397,151.0,0.016342,1.338332,543.0,0.019071
1659,2024-11-06,0.411276,175.350006,178.639999,175.039993,178.330002,178.330002,27061500.0,39.894333,0.026115,...,0.022788,1.083656,38.0,0.014597,1.123776,151.0,0.016520,1.391390,543.0,0.019105
1660,2024-11-07,0.612464,179.110001,182.580002,178.889999,182.279999,182.279999,16730400.0,39.906818,0.021216,...,0.020076,1.105392,39.0,0.014462,1.147205,152.0,0.016551,1.421176,543.0,0.019116
1661,2024-11-08,0.322299,182.000000,182.350006,179.570007,179.860001,179.860001,15021500.0,39.917835,0.016007,...,0.022584,1.088879,38.0,0.014580,1.130671,151.0,0.016543,1.401310,543.0,0.019120


In [10]:
df = df.dropna()

In [11]:
df

,Date,weighted compound sentiment score,Open,High,Low,Close,Adj Close,Volume,VWAP,log_diff_Open,...,Volatility_5,Close_Ratio_60,Trend_60,Volatility_60,Close_Ratio_250,Trend_250,Volatility_250,Close_Ratio_1000,Trend_1000,Volatility_1000
64,2014-03-26,-0.401900,28.941769,29.179876,28.181868,28.193575,28.124506,103586819.0,16.623033,-0.003428,...,0.010992,0.967636,30.0,0.013745,1.159434,126.0,0.013814,1.630885,517.0,0.016134
65,2014-04-03,0.750600,28.414488,29.283602,28.129271,28.409002,28.339405,101983228.0,16.634752,-0.051540,...,0.008786,0.974184,30.0,0.013615,1.158835,127.0,0.013748,1.635524,516.0,0.016126
66,2014-04-16,0.318200,27.075665,27.773746,26.926073,27.750811,27.682827,97865955.0,16.706051,0.011447,...,0.029658,0.958278,31.0,0.017058,1.119554,127.0,0.014345,1.586605,519.0,0.016119
67,2014-04-17,0.802700,27.365368,27.399775,26.484785,26.731607,26.666121,136190888.0,16.718720,0.010643,...,0.028183,0.924299,30.0,0.017700,1.077253,126.0,0.014553,1.527151,518.0,0.016164
68,2014-04-23,0.077200,26.616425,26.620415,26.240458,26.274862,26.210493,41046384.0,16.731303,0.009695,...,0.028470,0.911138,31.0,0.017311,1.055654,125.0,0.014615,1.497657,517.0,0.016174
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1657,2024-11-04,0.353798,171.240005,171.919998,169.485001,170.679993,170.679993,16194000.0,39.866061,-0.001750,...,0.019914,1.039259,37.0,0.013825,1.077853,151.0,0.016346,1.333506,542.0,0.019070
1658,2024-11-05,0.165292,170.830002,172.097000,170.369995,171.410004,171.410004,12518300.0,39.874694,-0.002397,...,0.018507,1.042915,38.0,0.013779,1.081397,151.0,0.016342,1.338332,543.0,0.019071
1659,2024-11-06,0.411276,175.350006,178.639999,175.039993,178.330002,178.330002,27061500.0,39.894333,0.026115,...,0.022788,1.083656,38.0,0.014597,1.123776,151.0,0.016520,1.391390,543.0,0.019105
1660,2024-11-07,0.612464,179.110001,182.580002,178.889999,182.279999,182.279999,16730400.0,39.906818,0.021216,...,0.020076,1.105392,39.0,0.014462,1.147205,152.0,0.016551,1.421176,543.0,0.019116


In [57]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import numpy as np

# Assuming `df` is your DataFrame
features = ['weighted compound sentiment score','Log_Returns','Close_Ratio_2', 'Trend_2', 'Close_Ratio_5', 'Trend_5','Close_Ratio_60', 'Trend_60', 'Volatility_60',
       'Close_Ratio_250', 'Trend_250', 'Volatility_250', 'Close_Ratio_1000',
       'Trend_1000', 'Volatility_1000']
target = ['Target']
# Split data into training (80%) and testing (20%) sets
split_index = int(len(df) * 0.8)
train_data = df.iloc[:split_index]
test_data = df.iloc[split_index:]

# Separate features (X) and target (y)
X_train = train_data[features]
y_train = train_data[target]
X_test = test_data[features]
y_test = test_data[target]

# Initialize and train the Random Forest model
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)

# Predict probabilities for the test set
y_prob = rf_model.predict_proba(X_test)[:, 1]  # Probability for the positive class

# Custom threshold for classification
custom_threshold = 0.5  # Change this to your desired threshold
y_pred = (y_prob >= custom_threshold).astype(int)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Print evaluation results
print(f"Accuracy: {accuracy:.2f}")
print("\nConfusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(class_report)

# Optional: Display feature importances
feature_importances = pd.DataFrame({
    'Feature': features,
    'Importance': rf_model.feature_importances_
}).sort_values(by='Importance', ascending=False)

print("\nFeature Importances:")
print(feature_importances)


C:\Users\yaman\AppData\Local\Temp\ipykernel_27812\4262160353.py:24: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_model.fit(X_train, y_train)


Accuracy: 0.57

Confusion Matrix:
[[ 44  59]
 [ 53 104]]

Classification Report:
              precision    recall  f1-score   support

         0.0       0.45      0.43      0.44       103
         1.0       0.64      0.66      0.65       157

    accuracy                           0.57       260
   macro avg       0.55      0.54      0.55       260
weighted avg       0.56      0.57      0.57       260


Feature Importances:
                              Feature  Importance
0   weighted compound sentiment score    0.091215
4                       Close_Ratio_5    0.089006
12                   Close_Ratio_1000    0.082612
8                       Volatility_60    0.081302
2                       Close_Ratio_2    0.080507
14                    Volatility_1000    0.080429
6                      Close_Ratio_60    0.078955
11                     Volatility_250    0.078063
1                         Log_Returns    0.077022
9                     Close_Ratio_250    0.075984
10                  

In [55]:
# Count the number of 1.0 and 0.0 in y_pred
count_1 = np.sum(y_test == 1)
count_0 = np.sum(y_test == 0)

print(f"Number of 1.0: {count_1}")
print(f"Number of 0.0: {count_0}")


Number of 1.0: 157
Number of 0.0: 103


In [43]:
157/(157+103)

0.6038461538461538

In [59]:
features = ['Log_Returns','Close_Ratio_2', 'Trend_2', 'Close_Ratio_5', 'Trend_5','Close_Ratio_60', 'Trend_60', 'Volatility_60',
       'Close_Ratio_250', 'Trend_250', 'Volatility_250', 'Close_Ratio_1000',
       'Trend_1000', 'Volatility_1000']
target = ['Target']


import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import numpy as np

# Assuming `df` is your DataFrame
features = features
target = 'Target'

# Split data into training (80%) and testing (20%) sets
split_index = int(len(df) * 0.8)
train_data = df.iloc[:split_index]
test_data = df.iloc[split_index:]

# Separate features (X) and target (y)
X_train = train_data[features]
y_train = train_data[target]
X_test = test_data[features]
y_test = test_data[target]

# Initialize and train the Random Forest model
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)

# Predict probabilities for the test set
y_prob = rf_model.predict_proba(X_test)[:, 1]  # Probability for the positive class

# Custom threshold for classification
custom_threshold = 0.5  # Change this to your desired threshold
y_pred = (y_prob >= custom_threshold).astype(int)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Print evaluation results
print(f"Accuracy: {accuracy:.2f}")
print("\nConfusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(class_report)

# Optional: Display feature importances
feature_importances = pd.DataFrame({
    'Feature': features,
    'Importance': rf_model.feature_importances_
}).sort_values(by='Importance', ascending=False)

print("\nFeature Importances:")
print(feature_importances)


Accuracy: 0.56

Confusion Matrix:
[[55 48]
 [66 91]]

Classification Report:
              precision    recall  f1-score   support

         0.0       0.45      0.53      0.49       103
         1.0       0.65      0.58      0.61       157

    accuracy                           0.56       260
   macro avg       0.55      0.56      0.55       260
weighted avg       0.58      0.56      0.57       260


Feature Importances:
             Feature  Importance
3      Close_Ratio_5    0.096989
7      Volatility_60    0.090628
5     Close_Ratio_60    0.090276
11  Close_Ratio_1000    0.088441
1      Close_Ratio_2    0.088146
8    Close_Ratio_250    0.087606
0        Log_Returns    0.086837
10    Volatility_250    0.086803
13   Volatility_1000    0.083011
9          Trend_250    0.053312
12        Trend_1000    0.050727
6           Trend_60    0.047577
4            Trend_5    0.032033
2            Trend_2    0.017616
